In [4]:
import pandas as pd
import datetime as dt
from datetime import datetime
import numpy as np


In [7]:
pd.set_option('display.max_rows', None)


In [8]:
import json

# Abra o arquivo JSON e leia seu conteúdo com a codificação 'utf-8-sig'
with open('C:/Users/Pc-Server/Documents/GitHub/analyser/data/municipios.json', 'r', encoding='utf-8-sig') as arquivo_json:
    dados = json.load(arquivo_json)
df = pd.DataFrame(dados)
# 'dados' agora contém o conteúdo do JSON como um dicionário em Python
df.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [11]:
# Abra o arquivo JSON e leia seu conteúdo com a codificação 'utf-8-sig'
with open('C:/Users/Pc-Server/Documents/GitHub/analyser/data/estados.json', 'r', encoding='utf-8-sig') as arquivo_json:
    dados = json.load(arquivo_json)
est = pd.DataFrame(dados)
# 'dados' agora contém o conteúdo do JSON como um dicionário em Python
est.rename(columns={'nome':'estado'}, inplace=True)


In [12]:
df = df.merge(est[['uf', 'estado', 'codigo_uf']], on= 'codigo_uf')

In [13]:
import unicodedata

# Suponha que você já tenha um DataFrame chamado df

# Função para remover acentos
def remover_acentos(texto):
    return ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

# Função para processar a coluna "nome"
def processar_coluna_nome(nome):
    # Remove acentos
    nome_sem_acentos = remover_acentos(nome)
    # Converte para maiúsculas
    nome_maiusculo = nome_sem_acentos.upper()
    # Substitui apóstrofos por espaços
    nome_processado = nome_maiusculo.replace("'", " ")
    return nome_processado

# Aplica a função de processamento à coluna "nome"
df['nome'] = df['nome'].apply(processar_coluna_nome)
df['estado'] = df['estado'].apply(processar_coluna_nome)

df.rename(columns={'nome': 'cidade'}, inplace=True)

In [14]:
df.to_csv('data/cidades.csv', sep= ';', index= False)